In [1]:
%cd tgi

/home/ubuntu/rshaw/BLoRA-TGI/tgi


In [2]:
%load_ext autoreload
%autoreload 2

import torch
from service.blora_utils import load_loras, prepare_batch
from transformers import LlamaForCausalLM, LlamaTokenizer
from IPython.display import clear_output

# from peft.tuners.lora import Linear
# from blora_utils import forward
# Linear.forward = forward

torch.set_default_tensor_type(torch.cuda.HalfTensor)

/home/ubuntu/rshaw/BLoRA-TGI/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_path = "decapoda-research/llama-7b-hf"
model = LlamaForCausalLM.from_pretrained(model_path, device_map="auto")
tokenizer = LlamaTokenizer.from_pretrained(model_path)
tokenizer.pad_token = 0

Loading checkpoint shards: 100%|██████████| 33/33 [00:08<00:00,  3.86it/s]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
loras = ["jondurbin/airoboros-7b-gpt4-1.2-peft", "trl-lib/llama-7b-se-rl-peft", "winddude/wizardLM-LlaMA-LoRA-7B"]
model, lora_map = load_loras(model, loras)

In [5]:
model.training

False

In [59]:
inputs = [('Outline a five sentence short story where a character stumbles upon a secret room in their house that contains relics from their future.',
  'jondurbin/airoboros-7b-gpt4-1.2-peft'),
#  ('Write a 6 line dialogue between a character and a magical creature that only they can see.',
#   'trl-lib/llama-7b-se-rl-peft'),
#  ('Describe a four sentence scene where a character discovers a hidden talent that changes their life forever.',
#   'winddude/wizardLM-LlaMA-LoRA-7B'),
#  ('Sculpt a three verse poem about the feeling of walking through a lush, vibrant garden in full bloom.',
#   'trl-lib/llama-7b-se-rl-peft'),
#  ('Develop an eight sentence short story about a character who can bring their dreams into reality, but only for a limited time.',
#   'winddude/wizardLM-LlaMA-LoRA-7B')
]

batch_lora_ids = [inp[1] for inp in inputs]

batch = prepare_batch(inputs, tokenizer, model, lora_map)

In [60]:
batch.input_ids.shape

torch.Size([1, 30])

In [125]:
outputs = []

for out in model.generate(
    **batch,
    max_length=100,
    stream_output=True
):
    outputs.append(out)
    batch_decoded = tokenizer.batch_decode(torch.cat([out.reshape(-1, 1) for out in outputs], dim=1))
    clear_output(wait=True)
    print("\n\n".join([lora + ":\n" + prompt + '\n' + decoded for (prompt, lora), decoded in zip(inputs, batch_decoded)]))

jondurbin/airoboros-7b-gpt4-1.2-peft:
Outline a five sentence short story where a character stumbles upon a secret room in their house that contains relics from their future.

The character, who is a young boy named Timmy, stumbles upon a secret room in his house that contained relics from his future. The room was hidden behind a bookcase in the library, and it was filled with strange artifacts and documents.
Timmy's curiosity got the best of him, and he decided


In [68]:
batch.input_ids.shape

torch.Size([1, 30])

[0, 4451, 1220, 263, 5320, 10541, 3273, 5828, 988, 263, 2931, 380, 3774, 793, 2501, 263, 7035, 5716, 297, 1009, 3699, 393, 3743, 337, 506, 29879, 515, 1009, 5434, 29889]
tensor([[    0,  4451,  1220,   263,  5320, 10541,  3273,  5828,   988,   263,
          2931,   380,  3774,   793,  2501,   263,  7035,  5716,   297,  1009,
          3699,   393,  3743,   337,   506, 29879,   515,  1009,  5434, 29889]])


In [137]:
tokens = batch.input_ids[0,:].tolist()
print(tokens)

model_kwargs = {
    "attention_mask": batch.attention_mask,
    "use_cache": True
}

input_ids = batch.input_ids

for _ in range(5):
    model_inputs = model.prepare_inputs_for_generation(input_ids, **model_kwargs)

    for key in model_inputs:
        if key != "past_key_values":
            print(f"{key}: {model_inputs[key]}")
    
    outputs = model(
        **model_inputs,
        return_dict=True,
        output_attentions=False,
        output_hidden_states=False,
    )

    next_token_logits = outputs.logits[:, -1, :]
    next_tokens = torch.argmax(next_token_logits, dim=-1)
    tokens.append(next_tokens.item())

    input_ids = torch.cat([batch.input_ids, next_tokens[:, None]], dim=-1)
    model_kwargs = model._update_model_kwargs_for_generation(
        outputs, model_kwargs, is_encoder_decoder=False
    )

    print(next_tokens.item())
    print("\n\n")
    # print(len(tokens))
    # print(tokenizer.decode(tokens[1:]))

[0, 4451, 1220, 263, 5320, 10541, 3273, 5828, 988, 263, 2931, 380, 3774, 793, 2501, 263, 7035, 5716, 297, 1009, 3699, 393, 3743, 337, 506, 29879, 515, 1009, 5434, 29889]
input_ids: tensor([[    0,  4451,  1220,   263,  5320, 10541,  3273,  5828,   988,   263,
          2931,   380,  3774,   793,  2501,   263,  7035,  5716,   297,  1009,
          3699,   393,  3743,   337,   506, 29879,   515,  1009,  5434, 29889]])
position_ids: tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])
use_cache: True
attention_mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])
13



input_ids: tensor([[13]])
position_ids: tensor([[30]])
use_cache: True
attention_mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])
1576



input_ids: tensor([[1576]])
position_ids: tensor([[31]])
use_cache: Tr

In [121]:
print(len(model_kwargs["past_key_values"]))
print(len(model_kwargs["past_key_values"][0]))
print(model_kwargs["past_key_values"][0][0].shape)
print(model_kwargs["past_key_values"][0][0][0,0,0,0])

32
2
torch.Size([1, 32, 30, 128])
tensor(-0.0251)


In [101]:
input_ids

tensor([[    0,  4451,  1220,   263,  5320, 10541,  3273,  5828,   988,   263,
          2931,   380,  3774,   793,  2501,   263,  7035,  5716,   297,  1009,
          3699,   393,  3743,   337,   506, 29879,   515,  1009,  5434, 29889,
          1576]])

In [78]:
outputs.keys()

odict_keys(['logits', 'past_key_values'])

In [17]:
input_ids = batch.input_ids
attention_mask = batch.attention_mask

In [8]:
model

StreamingPeftModel(
  (base_model): BLoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): BLinear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (jondurbin/airoboros-7b-gpt4-1_2-peft): Dropout(p=0.05, inplace=False)
                  (trl-lib/llama-7b-se-rl-peft): Dropout(p=0.05, inplace=False)
                  (winddude/wizardLM-LlaMA-LoRA-7B): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (jondurbin/airoboros-7b-gpt4-1_2-peft): Linear(in_features=4096, out_features=64, bias=False)
                  (trl-lib/llama-7b-se-rl-peft): Linear(in_features=4096, out_features=16, bias=False)
                  (winddude/wizardLM-LlaMA-LoRA-7B